# 使用Seq2Seq进行机器翻译

# 目录

* [1. 文本数据预处理](#1)
* [2. 创建批量数据](#2)
* [3. 构建含注意力机制的Seq2Seq模型](#3)
    * [3.1 构造编码器](#3.1)
    * [3.2 使用Bahdanau Attention机制](#3.2)
    * [3.3 构造解码器](#3.3)
* [4. 训练模型](#4)
    * [4.1 定义优化方法和损失函数](#4.1)
    * [4.2 定义训练步骤](#4.2)
    * [4.3 训练模型](#4.3)
* [5. 翻译句子](#5)

在这里将使用keras框架来搭建一个基于Attention的机器翻译模型。 本案例数据（原始下载路径）包含了19577条中文-英文的翻译句子对。 每一行代表了一个翻译对，并使用制表符tab作为中文（繁体字）和英文的分割。

# 1. 文本数据预处理
<a id=1></a>

In [2]:
import os
import re
import jieba
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
num_samples = 10000  #选取num_samples个样本进行训练

# 数据路径
data_path = './eng-chi.txt'

# 读取数据
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
print("文本中一共有%s对文本。"%(len(lines)))

文本中一共有19578对文本。


因为在英文中需要需要保留英文符号，而分词是按照空格分词的，所以需要在分词前把标点符号前加一个空格。同时还要添加标志序列开始和结束的字符，中文使用jieba库分词，然后使用空格隔开，所以创建如下函数：

In [4]:
#处理英文句子函数
def preprocess_engtext(w):
    # 在单词与跟在其后的标点符号之间插入一个空格，在分词的时候可以把标点符号也考虑进去
    # 例如： "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,])", r" \1 ", w)#ps：很神奇
    w = re.sub(r'[" "]+', " ", w)
    # 除了 (a-z, A-Z, ".", "?", "!", ","，""'")，将所有字符替换为空格
    w = re.sub(r"[^a-zA-Z?.!,']+", " ", w)
    #删除结尾和开头的空格
    w = w.strip()
    # 给句子加上开始和结束标记
    # 以便模型知道何时开始和结束预测
    w = '<start> ' + w + ' <end>'
    return w

#同样的处理中文句子函数
def preprocess_chitext(w):
    #清除多余空格
    w = re.sub(r'[" "]+', "", w)
    #使用jieba库分词
    w = " ".join(jieba.lcut(w))
    w = '<start> ' + w + ' <end>'
    return w

In [5]:
# 储存文本，输入样本为英文，输出样本为中文
input_texts = []
target_texts = []

#从lines中取num_samples个样本,并进行预处理
for line in lines[: min(num_samples, len(lines) - 1)]:    
    input_text, target_text = line.split('\t')
    #对中英文样本进行预处理
    input_text = preprocess_engtext(input_text)
    target_text = preprocess_chitext(target_text)
    input_texts.append(input_text)
    target_texts.append(target_text)

# 统计英文文本的最大序列长度。
max_encoder_seq_length = max([len(txt.split()) for txt in input_texts])
# 统计中文文本的最大序列长度。
max_decoder_seq_length = max([len(txt.split()) for txt in target_texts])

print('实际输入文本的句子的数量：', len(input_texts))
print('输入的最大文本长度：', max_encoder_seq_length)
print('输出最大文本的长度：', max_decoder_seq_length)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.771 seconds.
Prefix dict has been built successfully.


实际输入文本的句子的数量： 10000
输入的最大文本长度： 12
输出最大文本的长度： 15


对句子进行补齐可以选用各自最大的文本长度。下面查看一下得到的样本：

In [6]:
print(input_texts[-1])
print(target_texts[-1])

<start> I agree with you on this issue . <end>
<start> 我 同意 你 對 這問題 的 看法 。 <end>


下面对文本进行分词处理，使用keras的Tokenizer方法创建函数：

In [7]:
def tokenize(texts):
    tokenizer = Tokenizer(filters='')#不要把标点符号也给过滤掉了
    tokenizer.fit_on_texts(texts)
    tensor = tokenizer.texts_to_sequences(texts)
    tensor = pad_sequences(tensor, padding='post') #这里都选择往后填充，因为后面会用到将输入数据逆序
    #返回处理好的序列和字典
    return tensor, tokenizer

In [8]:
input_data, input_tokenizer = tokenize(input_texts)
target_data, target_tokenizer = tokenize(target_texts)

得到两个文本的整数序列索引：

In [9]:
input_data[-1],target_data[-1]

(array([   1,    4,  358,   49,    5,   35,   19, 3566,    3,    2,    0,
           0], dtype=int32),
 array([   1,    4,  220,    6,   73, 7081,    5,  748,    3,    2,    0,
           0,    0,    0,    0], dtype=int32))

In [10]:
input_data.shape, target_data.shape

((10000, 12), (10000, 15))

# 2. 创建批量数据
<a id=2></a>

In [12]:
BATCH_SIZE = 64   #生成批量数据集的批次大小
steps_per_epoch = len(input_data)//BATCH_SIZE
embedding_dim = 100 #词嵌入维度
units = 512   #GRU隐层神经元个数
input_dim = len(input_tokenizer.word_index) + 1    #英文字典的大小
target_dim = len(target_tokenizer.word_index) + 1  #中文字典的大小
#生成一个产生批量数据的迭代器
dataset = tf.data.Dataset.from_tensor_slices((input_data, target_data)).shuffle(len(input_data))
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

比如从迭代器中生成一个`BATCH_SIZE`大小的数据集：

In [13]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 12]), TensorShape([64, 15]))

# 3. 构建含注意力机制的Seq2Seq模型
<a id=3></a>

## 3.1 构造编码器
<a id=3.1></a>

编码器使用GRU对文本进行编码，最后的一个输出作为解码器的输入，但同时也要记录每个时间步的输出，因为注意力机制生成的向量实际上就是这些输出的加权和。因此GRU层的参数中`return_sequences=True`可以得到每个时间步的输出，`return_state=True`会得到最后一个时间步的输出。本案例的模型构建部分使用的是tensorflow官方教程里的模型：https://www.tensorflow.org/tutorials/text/nmt_with_attention

In [14]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_dim, embedding_dim, enc_units, batch_sz):
        """
        vocab_dim: 输入英文数据的维度（也就是字典大小）
        embedding_dim: 词嵌入后的维度
        enc_units: 编码器隐层神经元数量
        batch_sz: 输入数据的样本数，这里用于初始化GRU的初始状态
        """
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = Embedding(vocab_dim, embedding_dim)
        self.gru = GRU(self.enc_units,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')
        
    def call(self, x, hidden):
        """
        x : 输入数据，2维张量（批量大小，序列长度），因为有embedding层
        
        """
        x = self.embedding(x)
        output, state = self.gru(x,initial_state = hidden)
        return output, state   
        # output 为每个时间步的输出，是个3维张量(批量大小, 序列长度（时间步）, 神经元数量)，
        # state为最后一个时间步的输出（向量），是2维张量(批量大小, 神经元数量) 
        
    def initialize_hidden_state(self):
        #初始化开始的隐层状态
        return tf.zeros((self.batch_sz, self.enc_units))

使用上次创建好的数据样例进行初步测试，检查输出的维度是否正确：

In [15]:
encoder = Encoder(input_dim, embedding_dim, units, BATCH_SIZE)
# 样本输入
sample_state = encoder.initialize_hidden_state()
sample_output, sample_state = encoder(example_input_batch, sample_state)
print ('编码器的第一个输出维度: {} (批量大小, 序列长度（时间步）, 神经元数量) '.format(sample_output.shape))
print ('编码器的第二个输出维度: {}(批量大小, 神经元数量) '.format(sample_state.shape))

编码器的第一个输出维度: (64, 12, 512) (批量大小, 序列长度（时间步）, 神经元数量) 
编码器的第二个输出维度: (64, 512)(批量大小, 神经元数量) 


## 3.2 使用Bahdanau Attention机制
<a id=3.2></a>

先简要介绍一下Bahdanau Attention机制的公式，设以解码器在时间步$t'$的隐藏状态$s_{t'}$与编码器在时间步$t$的隐藏状态$h_t$为输入，得到权重：

$$score(s_{t'},h_{t})= v^{T}tanh(W_1s_{t'}+W_2h_{t})$$

$$\alpha_{tt'} = \frac{exp(score(h_{t},s_{t'}))}{\sum_{t''}exp(score(h_{t},s_{t''}))}$$

其中$v$，$W_1$，$W_2$都是需要学习的参数，事实上可以使用Dense Layer表示矩阵相乘，并且最后权重计算可以使用Softmax输出。

最后得到的参与解码层$t'$时刻的输入的Attention向量是：

$$c_{t'} = \sum_t \alpha_{tt'}h_t$$

In [16]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        #使用Dense layer来表示公式中的矩阵乘法， 
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)
        #事实上，units是神经元数量，也代表公式中向量v的长度

    def call(self, dec_state, enc_output):
        """
        dec_state: 解码器某个时间的隐层状态，是二维的张量（批量大小，隐藏层大小）
        enc_output: 编码器的全部时间的输出，是三维的张量（批量大小，序列长度，隐藏层大小）
        """
        # hidden_with_time_axis 的形状 == （批大小，1，隐藏层大小）
        # 这样做是为了方便在加法中使用广播机制以计算分数score
        hidden_with_time_axis = tf.expand_dims(dec_state, 1)

        # 最后得到的分数score的形状 == （批大小，编码器序列长度，1）
        # 我们在最后一个轴上得到 1， 因为我们把分数应用于 self.V
        # 在应用 self.V 之前，张量的形状是（批大小，编码器序列长度，Dense Layer大小），这里用了广播机制
        score = self.V(tf.nn.tanh(
            self.W1(enc_output) + self.W2(hidden_with_time_axis)))

        # 注意力权重 （attention_weights） 的形状 == （批大小，编码器序列长度，1）最后这一维向量便是编码器每个时间步对应的权重，是标量
        attention_weights = tf.nn.softmax(score, axis=1)

        # 上下文向量 （context_vector） 求和之后的形状 == （批大小，编码器隐藏层大小）
        # 这里乘法也在最后一维上使用了广播机制
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

使用样例进行测试，得到输出的形状与代码中分析的一致。

In [17]:
attention_layer = BahdanauAttention(10) # 10是公式中向量v的大小
attention_result, attention_weights = attention_layer(sample_state, sample_output)
# sample_state事实上是解码器的初始状态，可以作为Attention的第一个输入

print("上下文向量的形状:  {}   (batch size, units)".format(attention_result.shape))
print("注意力权重的形状:  {} (batch_size, sequence_length, 1)".format(attention_weights.shape))

上下文向量的形状:  (64, 512)   (batch size, units)
注意力权重的形状:  (64, 12, 1) (batch_size, sequence_length, 1)


## 3.3 构造解码器
<a id=3.3></a>

我们直接将编码器在最终时间步的隐藏状态作为解码器的初始隐藏状态。这要求编码器和解码器的循环神经网络使用相同的隐藏层个数和隐藏单元个数。

在解码器的前向计算中，通过刚刚构造的注意力机制计算得到当前时间步的上下文向量。由于解码器的输入来自输出语言的文本向量，我们将输入通过词嵌入层得到低维的文本向量，然后和上下文向量（context_vector）在特征维连结。我们将连结后的结果与上一时间步的隐藏状态通过门控循环单元计算出当前时间步的输出与隐藏状态。最后，我们将输出通过全连接层变换为有关各个输出词的预测。

需要注意的是，我们训练的时候，并不是一次性把整个输出文本的序列输入到解码器，而是逐个输入到解码器，得到输出后在输入到下一个解码器，因此每个解码器GRU的序列长度为1，输入文本的形状为（批量大小，1），但是通过多层解码器的叠加，也实现了文本序列长度的GRU模型。

In [22]:
class Decoder(tf.keras.Model):
        def __init__(self, vocab_dim, embedding_dim, dec_units, batch_sz):
            """
            vocab_dim： 中文文本词典的维度
            embedding_dim： 同编码器，嵌入层使用这两个维度来学习词向量
            dec_units: 解码器GRU层神经元数量
            batch_sz:  批量大小
            """
            super(Decoder, self).__init__()
            self.batch_sz = batch_sz
            self.dec_units = dec_units
            self.embedding = Embedding(vocab_dim, embedding_dim)
            self.gru = GRU(self.dec_units,
                            return_sequences=True,
                            return_state=True,
                            recurrent_initializer='glorot_uniform')
            
            self.fc = Dense(vocab_dim)
            #使用注意力机制
            self.attention = BahdanauAttention(self.dec_units)

        def call(self, x, hidden, enc_output):
            """
            x: 批量数据， 输入形状为（批量大小，1），时间步为1，因为解码需要一个一个输出
            hidden: 这里输入上一个解码器的状态，是二维张量（批量大小，解码器隐藏层大小）
            enc_output: 编码器的输出（attention用），是三维张量（批量大小，编码器序列长度，编码器隐藏层大小）
            """
            # 得到上下文向量和注意力权重
            context_vector, attention_weights = self.attention(hidden, enc_output)
            # 词嵌入
            x = self.embedding(x)
            # 将文本数据的向量与上下文向量一起输入GRU层，这时输入向量的维度为（批量大小，1， 嵌入维度 + 编码器隐藏层大小）
            x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
            # 得到gru的输出，这里由于输入的序列是1，所以output的形状为（批量大小，1，隐藏层大小）
            # state的形状为（批量大小，隐藏层大小），这里需要返回state作为下一个attention的输入
            output, state = self.gru(x)
            # 将时间步的维度去掉，然后送入全连接层进行预测
            output = tf.reshape(output, (-1, output.shape[2]))
            x = self.fc(output)

            return x, state, attention_weights

测试样例：

In [23]:
decoder = Decoder(target_dim, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_state, sample_output)

print ('解码器输出形状: {} (批量大小, 中文字典大小)'.format(sample_decoder_output.shape))

解码器输出形状: (64, 7082) (批量大小, 中文字典大小)


# 4. 训练模型
<a id=4></a>

## 4.1 定义优化方法和损失函数
<a id=4.1></a>

In [25]:
optimizer = Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    #真实值中等于0的为false，不等于0的为True
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    
    loss_ = loss_object(real, pred)
    #文本padding后的0不参与计算损失
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [26]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## 4.2 定义训练步骤
<a id=4.2></a>

1. 将文本输入向量传送至编码器，编码器返回每个时间步的输出和编码器最后一步的隐藏层状态。
2. 将编码器输出、编码器隐藏层状态和解码器输入（即开始标记）传送至解码器。
3. 解码器返回预测和解码器隐藏层状态。
4. 解码器隐藏层状态被传送回模型，预测被用于计算损失。
5. 使用教师强制 （teacher forcing） 决定解码器的下一个输入。
6. 教师强制指的是将真实的输出文本向量中的下一个词向量作为下一个输入传送至解码器。
7. 最后一步是计算梯度，并将其应用于优化器和反向传播。

In [27]:
def train_step(inp, targ, enc_hidden):
    """
    inp : 输入文本向量，为二维张量（批量大小，序列长度）
    targ： 输出文本向量，为二维张量（批量大小，序列长度）
    enc_hidden： 编码器初始状态
    """
    loss = 0

    with tf.GradientTape() as tape:
        # 将输入文本向量传送至编码器，编码器返回编码器每个时间步的输出和编码器最后一步的隐藏层状态
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        # 将编码器的最后一步隐层状态作为输入解码器的初始隐层状态
        dec_hidden = enc_hidden
        # 标识符<start>作为解码器第一个输入的文本向量
        dec_input = tf.expand_dims([target_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

        # 教师强制 - 将真实的输出文本向量（targ）中的下一个词向量作为下一个输入
        for t in range(1, targ.shape[1]):
            # 将编码器输出 （enc_output） 上一个解码器的隐层状态（dec_hidden）传送至解码器
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            # 计算loss
            loss += loss_function(targ[:, t], predictions)
            # 使用教师强制
            dec_input = tf.expand_dims(targ[:, t], 1)
    #每一批次的损失
    batch_loss = (loss / int(targ.shape[1]))
    #训练变量
    variables = encoder.trainable_variables + decoder.trainable_variables
    #计算梯度
    gradients = tape.gradient(loss, variables)
    #使用Adam优化算法计算梯度
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

## 4.3 训练模型
<a id=4.3></a>
使用定义好的训练函数训练数据，这里实际运行了50个epochs，但只显示最后10个epochs达到的效果。

In [75]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    # 从创建好的dataset中抽取最多steps_per_epoch个数据
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

    # 每 2 个周期（epoch），保存（检查点）一次模型
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch: {}/{}  Time: {:.0f}s  Loss: {:.4f}\n  '.format(epoch + 1, EPOCHS, time.time() - start,
                          total_loss / steps_per_epoch))

Epoch: 1/10  Time: 31s  Loss: 0.0723
  
Epoch: 2/10  Time: 31s  Loss: 0.0653
  
Epoch: 3/10  Time: 31s  Loss: 0.0626
  
Epoch: 4/10  Time: 31s  Loss: 0.0552
  
Epoch: 5/10  Time: 31s  Loss: 0.0450
  
Epoch: 6/10  Time: 32s  Loss: 0.0377
  
Epoch: 7/10  Time: 31s  Loss: 0.0341
  
Epoch: 8/10  Time: 31s  Loss: 0.0297
  
Epoch: 9/10  Time: 32s  Loss: 0.0272
  
Epoch: 10/10  Time: 31s  Loss: 0.0271
  


# 5. 翻译句子
<a id=5></a>

翻译句子的步骤与训练时的步骤基本一致，只是要重新对句子做预处理，而且不使用教学机制，即以解码器的输出作为下一个解码器的输入。

In [76]:
def translate2chinese(sentence):
    """
    sentence: 输入一个英文句子字符串
    """
    # 将输入的英文句子作同样的预处理，转换成整数序列
    sentence = preprocess_engtext(sentence)
    sentence = sentence.lower()
    inputs = [input_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = pad_sequences([inputs], maxlen=input_data.shape[1], padding='post')
    inputs = tf.convert_to_tensor(inputs)
    # 输出结果
    result = ''
    # 输入到编码器
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    # 输入到解码器，这里不使用教学机制
    for t in range(target_data.shape[1]):
        predictions, dec_hidden, _ = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # 得到预测值
        predicted_id = tf.argmax(predictions[0]).numpy()
        # 根据预测值生成文本
        result += target_tokenizer.index_word[predicted_id] + ' '
        # 若预测出结束标识符，停止进行下一步预测
        if target_tokenizer.index_word[predicted_id] == '<end>':
            break
        # 预测的 ID 被输送回模型
        dec_input = tf.expand_dims([predicted_id], 0)
    print('Input: %s' % (sentence))
    print('Translation: {}'.format(result))

In [82]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [83]:
translate2chinese("Have a nice day.")

Input: <start> have a nice day . <end>
Translation: 祝 你 一天 过得 愉快 。 <end> 


In [92]:
translate2chinese("I'll wait until four o'clock.")

Input: <start> i'll wait until four o'clock . <end>
Translation: 我會 等到 四點 。 <end> 


In [93]:
translate2chinese("I want to become an engineer.")

Input: <start> i want to become an engineer . <end>
Translation: 我 想要 成為 一位 工程 師 。 <end> 


也有一些句子翻译的还不好。这种句子并不是从单词推断意思，而是这整个句子是一个意思。

In [96]:
translate2chinese("How are you.")

Input: <start> how are you . <end>
Translation: 你 呢 嗎 ？ <end> 


由于数据只是一些简单的中英文本对，加之训练的数据非常少，并不能翻译稍复杂的句子，而且对于训练数据的翻译效果比较好，泛化性能却很差，但是这个模型还是可以自动生成比较通顺的句子。

In [104]:
translate2chinese("I would rather live by myself than do as he tells me to do.")

Input: <start> i would rather live by myself than do as he tells me to do . <end>
Translation: 我 非常 比 任何人 都 喜歡 網球 。 <end> 


诶，想可视化Attention机制也失败了，总之，结果不太理想，还是有很多细节需要修改啊！！！